# Explore Data

Source: https://archive.ics.uci.edu/ml/datasets/online+retail#

**Attribute Information**:

`InvoiceNo`: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

`StockCode`: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

`Description`: Product (item) name. Nominal.

`Quantity`: The quantities of each product (item) per transaction. Numeric.

`InvoiceDate`: Invice Date and time. Numeric, the day and time when each transaction was generated.

`UnitPrice`: Unit price. Numeric, Product price per unit in sterling.

`CustomerID`: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

`Country`: Country name. Nominal, the name of the country where each customer resides.

## Summary of Data Processing

Rows Removed:
- `UnitPrice` == 0
- `CustomerID` == NaN
- `Country` == `Unspecified`
- `StockCode` == `POST`, `BANK CHARGES`, `PADS`, `DOT`, `CRUK`
- `CustomerID`, where effective `Quantity` < 0

Others:
- Data type changed

# Set up

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Data

### Load from a Dataset registered in AML Workspace.

> Potential Bug: 
> - `online-retail.csv` is registered as dataset with both settings `properties` == `None` or `properties` == `date`. When it is loaded by the cell below using `azure.core.Dataset`, a large proportion of the column `InvoiceDate` containing dtype `datetime64[ns]` has become `NaT`. Refer to [00-explore-data-00.ipynb](./00-explore-data-00.ipynb)
> - see temporary mitigation in cell above.


In [ ]:
from azureml.core import Workspace, Dataset

workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep = '\n')

dataset = Dataset.get_by_name(workspace, name='online-retail')
df_orig = dataset.to_pandas_dataframe()

In [ ]:
df = df_orig.copy()
df

In [ ]:
CustomerID = 16923
StockCode = '21485'	

condition = (df['CustomerID']==CustomerID)
df[condition]

condition = (df['CustomerID']==CustomerID) & (df['Quantity']<0)
df[condition]
StockCode_returned = df[condition]['StockCode']

condition = (df['CustomerID']==CustomerID) & (df['StockCode']==StockCode)
df[condition]

for sc in StockCode_returned:
    condition = (df['CustomerID']==CustomerID) & (df['StockCode']==sc)
    df[condition]

## Explore Data

### Basic

In [ ]:
#df.describe(include='all', datetime_is_numeric=True)
df.describe(include='all')

In [ ]:
df.isnull().sum(axis=0) # axis = 0 refer to column-wise

In [ ]:
df.info()

In [ ]:
# change data type
df = df.astype({'StockCode' : 'category',
                'Country' : 'category',})

# convert the 'Date' column to datetime format
df['InvoiceDate']= pd.to_datetime(df['InvoiceDate'])

df.info()

### Correlation

In [ ]:
df.corr()

### Pair-plot

In [ ]:
df_pairplot = df.copy()
#df_pairplot = df.sample(frac=0.01, random_state=9) # 1% ~ 540 points

_ = sns.pairplot(df_pairplot, hue='Country', height=4, aspect=1.5); plt.show() # 

In [ ]:
df_pairplot = df.sample(frac=0.01, random_state=9) # 1% ~ 540 points
_ = sns.pairplot(df_pairplot, hue='Country', height=4, aspect=1.5); plt.show() # 

### Histogram : Numerical

In [ ]:
#df_hist = df[(df['Quantity']>=-10) & (df['Quantity']<=10)]
#df_hist.describe()

df_hist = df.copy()

bins=100

_ = df_hist[['Quantity', 'Country']].plot.hist(bins=bins, alpha=0.5, by='Country', figsize=(10,120)); plt.show()

_ = df_hist[['UnitPrice', 'Country']].plot.hist(bins=bins, alpha=0.5, by='Country', figsize=(10,120)); plt.show()

Note:
- Extreme value for `Quantity` and `UnitPrice`

In [ ]:
df.sort_values(by=['Quantity'])
df.sort_values(by=['Quantity']).head(10)
df.sort_values(by=['Quantity']).tail(10)

Note:
- `Quantity` negative, `InvoiceNo` no letter `C`, seems to mean stock adjustment, e.g. damaged, thrown away, etc
    - Investigate < TO DO >
- `UnitPrice` have value `0`. < REMOVED >
- `CustomerID` has `nan` , what does this mean? Customer who bought things but does not register? Remove for now. < REMOVED >
- rows where `InvoiceNo` has no letter `C`, and `Quantity` is `<0`, or `UnitPrice` is `0` < Unsure of what it means >  < SOLVED >

In [ ]:
df.sort_values(by=['UnitPrice'])
df.sort_values(by=['UnitPrice']).head(20)
df.sort_values(by=['UnitPrice']).tail(20)

Note
- `InvoiceNo` contain letter `A`, which is not in the Data Definition. Seems to mean `Adjust bad debt`, with `StockCode` `B`
- `Stockcode` that seems not to refer to a product, includes, but not limited to, `AMAZONFEE`, `M`, `B`, `POST`, `DOT`
    - Extract `StockCode` that contain letters to further understand < DONE >

To Clean
- Remove rows where is `UnitPrice` is `0` or `NaN`

## Remove unwanted data

### Remove rows where `CustomerID` is `nan`

In [ ]:
df.dropna(subset=['CustomerID'], inplace=True)
df.describe(include='all', datetime_is_numeric=True)
df.isnull().sum(axis=0) # axis = 0 refer to column-wise

In [ ]:
# change data type to int, then str, due to decimal point, e.g. 1234.0
df = df.astype({'CustomerID' : int})  
df = df.astype({'CustomerID' : str})
df.describe(include='all', datetime_is_numeric=True)

### Normalise Text

In [ ]:
df['Description'] = df['Description'].str.lower()
df

### After removing `CustomerID`==`NaN` : Histogram

In [ ]:
#df_hist = df[(df['Quantity']>=-10) & (df['Quantity']<=10)]
#df_hist.describe()

df_hist = df.copy()

bins=100

_ = df_hist[['Quantity', 'Country']].plot.hist(bins=bins, alpha=0.5, by='Country', figsize=(10,120)); plt.show()

_ = df_hist[['UnitPrice', 'Country']].plot.hist(bins=bins, alpha=0.5, by='Country', figsize=(10,120)); plt.show()

In [ ]:
#df.sort_values(by=['Quantity'])
df.sort_values(by=['Quantity']).head(50)
df.sort_values(by=['Quantity']).tail(50)

In [ ]:
#df.sort_values(by=['UnitPrice'])
df.sort_values(by=['UnitPrice']).head(50)
df.sort_values(by=['UnitPrice']).tail(50)

### Remove rows where `Country`==`Unspecified`

In [ ]:
df.shape
df = df[df['Country']!='Unspecified']
df.shape

### Check Conditions Below:

- `UnitPrice` have value `0`
- `InvoiceNo` has no letter `C`, and `Quantity` is `<0`, or `UnitPrice` is `0` < Unsure of what it means >  < Solved >

In [ ]:
df[df['InvoiceNo'].str.contains('[a-zA-Z]')] # contain any alphabets

In [ ]:
df_temp = df['InvoiceNo'].str.replace('\d+', '') # replace all numeric with '', i.e. extract alphabets
df_temp.unique()

Note
- `InvoiceNo` now only contain numeric and `C` + numeric

In [ ]:
print('UnitPrice <= 0   AND   InvoiceNo contain letter C')
df[(df['UnitPrice']<=0) & (df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('UnitPrice < 0   AND   InvoiceNo contain letter C')
df[(df['UnitPrice']<0) & (df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('UnitPrice == 0   AND   InvoiceNo contain letter C')
df[(df['UnitPrice']==0) & (df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape

print('Quantity <= 0   AND   InvoiceNo contain letter C')
df[(df['Quantity']<=0) & (df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('Quantity < 0   AND   InvoiceNo contain letter C')
df[(df['Quantity']<0) & (df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('Quantity == 0   AND   InvoiceNo contain letter C')
df[(df['Quantity']==0) & (df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape

print('UnitPrice <= 0   AND   InvoiceNo not contain letter C')
df[(df['UnitPrice']<=0) & (~df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('UnitPrice < 0   AND   InvoiceNo not contain letter C')
df[(df['UnitPrice']<0) & (~df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('UnitPrice == 0   AND   InvoiceNo not contain letter C')
df[(df['UnitPrice']==0) & (~df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape

print('Quantity <= 0   AND   InvoiceNo not contain letter C')
df[(df['Quantity']<=0) & (~df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('Quantity < 0   AND   InvoiceNo not contain letter C')
df[(df['Quantity']<0) & (~df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape
print('Quantity == 0   AND   InvoiceNo not contain letter C')
df[(df['Quantity']==0) & (~df['InvoiceNo'].str.contains('[a-zA-Z]'))].shape

Note:
- 40 rows where `UnitPrice` == 0, `InvoiceNo` is numeric. What does this mean? Free gift? Remove for now. < DONE >
- rows where `InvoiceNo` contain letter `C`, AND `Quantity` is < 0. This is consistent now.

#### When `UnitPrice` == 0

In [ ]:
df[df['UnitPrice']==0]

### Remove when `UnitPrice` == 0

In [ ]:
df.shape
df.drop(df[df['UnitPrice']==0].index, inplace=True)
df.shape

### Investigate `StockCode`

In [ ]:
df_temp = df[df['StockCode'].str.contains('[a-zA-Z]')] # contain any alphabets
df_temp['StockCode'].unique()
df_temp

In [ ]:
df_temp = df['StockCode'].str.replace('\d+', '')
df_temp.unique()

In [ ]:
for alphabets in df_temp.unique():
    if alphabets: # if alphabets is not empty
        alphabets
        df[df['StockCode'].str.contains(alphabets)]

Note
- Remove `StockCode` == `POST`, `BANK CHARGES`, `PADS`, `DOT`, `CRUK` < DONE >
- Remove `UnitPrice < 0.01` < Gone when the above is done >

In [ ]:
df.shape

condition = (df['StockCode']=='POST') | \
            (df['StockCode']=='BANK CHARGES') | \
            (df['StockCode']=='PADS') | \
            (df['StockCode']=='DOT') | \
            (df['StockCode']=='CRUK')

df = df[~condition]
df.shape

#### Check Statistics

In [ ]:
df.describe(include='all', datetime_is_numeric=True)

Note
- extreme value for `Quantity` and `UnitPrice` still exist. < Investigate >

In [ ]:
df.sort_values(by=['Quantity']).head(50)
df.sort_values(by=['Quantity']).tail(50)

### Investigate extreme `Quantity` values 

Note
- Assume that we are only interested in effective sales, i.e. excluding return, remove pairs of (order, ordercancelled) < AS FEATURE ENGINEERING | TO DO >

In [ ]:
CustomerID_mini = df.sort_values(by=['Quantity']).head(3)['CustomerID']
CustomerID_mini = pd.concat([CustomerID_mini, df.sort_values(by=['Quantity']).tail(3)['CustomerID']])

df_mini = df[df['CustomerID'].isin(CustomerID_mini)]
df_mini.shape
df_mini

In [ ]:
df_mini_duplicated = df_mini[df_mini.duplicated(subset=['CustomerID', 'StockCode', 'Description', 'UnitPrice', 'Country'], keep=False)]
df_mini_duplicated

In [ ]:
condition = (df_mini_duplicated['StockCode']=='21787') & (df_mini_duplicated['CustomerID']=='12901')
df_mini_duplicated[condition]
df_mini_duplicated[condition]['Quantity'].sum()

In [ ]:
# observed=True : This only applies if any of the groupers are Categoricals. 
# If True: only show observed values for categorical groupers. 
# If False: show all values for categorical groupers.
df_mini_duplicated_sum = df_mini_duplicated.groupby(['CustomerID', 'StockCode', 'Description', 'UnitPrice', 'Country'], as_index=False, observed=True)['Quantity'].sum() 
df_mini_duplicated_sum

condition = (df_mini_duplicated_sum['StockCode']=='21787') & (df_mini_duplicated_sum['CustomerID']=='12901')
df_mini_duplicated_sum[condition]

In [ ]:
df_duplicated = df[df.duplicated(subset=['CustomerID', 'StockCode', 'UnitPrice', 'Country'], keep=False)]
df_effective_quantity = df_duplicated.groupby(['CustomerID', 'StockCode', 'UnitPrice', 'Country'], as_index=False, observed=True)['Quantity'].sum() 
df_effective_quantity.describe()
df_effective_quantity

In [ ]:
df_effective_quantity.sort_values(by=['Quantity']).head(50)
df_effective_quantity.sort_values(by=['Quantity']).tail(50)

In [ ]:
condition = (df_effective_quantity['Quantity']<0)
df_effective_quantity[condition]


Note
- effective `Quantity` < 0, what does this mean? Remove those `CustomerID`? Assuming return period of 12 months, will need 24 months data to evaluate customer value for last 12 months.
    - For simplicity, remove such `CustomerID`. < AS FEATURE PREPARATION | TO DO >

### Cross-checking

In [ ]:
CustomerID = '18139'

condition = (df['CustomerID']==CustomerID)
df[condition]

condition = (df['CustomerID']==CustomerID) & (df['Quantity']<0)
df[condition]

condition = (df['CustomerID']==CustomerID) & (df['UnitPrice']==0.32)
df[condition]

condition = (df['CustomerID']==CustomerID) & (df['UnitPrice']==0.56)
df[condition]

In [ ]:
CustomerID = '16923'
StockCode = '21485'	

condition = (df['CustomerID']==CustomerID)
df[condition]

condition = (df['CustomerID']==CustomerID) & (df['Quantity']<0)
df[condition]

condition = (df['CustomerID']==CustomerID) & (df['StockCode']==StockCode)
df[condition]

#### Investigate `UnitPrice` 

In [ ]:
df.sort_values(by=['UnitPrice']).head(50)
df.sort_values(by=['UnitPrice']).tail(50)

Note:
- Still exist extremly high `UnitPrice` < TO INVESTIGATE >

### Histogram : Categorical

#### `Country`

In [ ]:
categorical_cols = ['Country']

for col in categorical_cols:
    df[col].value_counts().plot(kind='bar', figsize=(15,4), title=col)
    plt.show()


In [ ]:
df.groupby('Country').size().sort_values(ascending=False)

In [ ]:
categorical_cols = df.select_dtypes(include=['category'])

for col in categorical_cols:
    df[col].value_counts(normalize=True).nlargest(100)

In [ ]:
for col in df.columns:
    col
    df[col].value_counts(normalize=True).nsmallest(100)

In [ ]:
for col in df.columns:
    col
    df[col].value_counts(normalize=True).nlargest(100)

### Scatter plot

In [ ]:
df_catplot = df.sample(frac=0.01, random_state=9) # 1% ~ 4000 points
_ = sns.catplot(x="Quantity", y="UnitPrice", hue="Country", data=df_catplot, height=5, aspect=3) # 2-3 min
_ = plt.xticks(rotation=90)

====================================================================================================

# Data Management

## Upload Processed Data to Datastore

In [ ]:
from azureml.core import Workspace, Dataset

workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep = '\n')

datastore = workspace.get_default_datastore()
datastore

if True:
# if False: # Replace `False` with `True` to run code below
    filename = '../../.aml/data/online-retail-processed.csv'

    # Save to local
    df.to_csv(filename, index=False)

    Dataset.File.upload_directory('../../.aml/data', datastore)

## Register Dataframe as Dataset

In [ ]:
from azureml.core import Workspace, Dataset

workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep = '\n')

datastore = workspace.get_default_datastore()
datastore

if True:
# if False: # Replace `False` with `True` to run code below

    # Dataset name to register as 
    name = 'online-retail-processed'

    # create a new dataset
    Dataset.Tabular.register_pandas_dataframe(dataframe=df, 
                                            target=datastore, 
                                            name=name, 
                                            show_progress=True, 
                                            tags={'Purpose':'Tutorial'})